In [1]:
import scanpy as sc
import sys
import pickle

sys.path.append("/data/work/tools")
import scCyclone as scc
import pandas as pd
import numpy as np
import scanpy as sc


     _______.  ______   ______ ____    ____  ______  __        ______   .__   __.  _______ 
    /       | /      | /      |\   \  /   / /      ||  |      /  __  \  |  \ |  | |   ____|
   |   (----`|  ,----'|  ,----' \   \/   / |  ,----'|  |     |  |  |  | |   \|  | |  |__   
    \   \    |  |     |  |       \_    _/  |  |     |  |     |  |  |  | |  . `  | |   __|  
.----)   |   |  `----.|  `----.    |  |    |  `----.|  `----.|  `--'  | |  |\   | |  |____ 
|_______/     \______| \______|    |__|     \______||_______| \______/  |__| \__| |_______|
                                                                                           
                                                                                                                                                       

Version: 1.0.3, Author: Dawn


## Generate adata

### Generate iso-level adata

In [2]:
isoquant_path="../test/input/test.transcript_model_grouped_counts.tsv"

In [3]:
adata_iso=scc.generate_Iso_adata(isoquant_path)
adata_iso

AnnData object with n_obs × n_vars = 2999 × 36848
    obs: 'batch'
    var: 'isoform'

#### Add sqanti3 result

In [4]:
sqanti_path="../test/input/test.extended_annotation_classification.txt"

In [5]:
scc.tl.add_sqanti3(adata_iso,sqanti_path)
adata_iso

AnnData object with n_obs × n_vars = 2999 × 36848
    obs: 'batch'
    var: 'isoform', 'chrom', 'strand', 'length', 'exons', 'structural_category', 'associated_gene', 'CDS_length', 'CDS_start', 'CDS_end', 'CDS_genomic_start', 'CDS_genomic_end', 'predicted_NMD'

#### gene_id translate gene symbol

In [6]:
gene_id_path="../test/input/gene_id_translate.txt"

In [7]:
gene_info=pd.read_csv(gene_id_path,sep=" ",header=None)
gene_info=gene_info.rename(columns={0:"associated_gene",1:"gene_name"})
gene_info

,associated_gene,gene_name
0,ENSDARG00000000001,slc35a5
1,ENSDARG00000000002,ccdc80
2,ENSDARG00000000018,nrf1
3,ENSDARG00000000019,ube2h
4,ENSDARG00000000068,slc9a3r1a
...,...,...
32009,ENSDARG00000117823,BX537296.5
32010,ENSDARG00000117824,CABZ01064670.1
32011,ENSDARG00000117825,CU207269.4
32012,ENSDARG00000117826,CR385041.2


In [8]:
adata_iso.var['gene_name']=list(pd.merge(adata_iso.var,gene_info,on="associated_gene",how="left")['gene_name'])

In [9]:
adata_iso

AnnData object with n_obs × n_vars = 2999 × 36848
    obs: 'batch'
    var: 'isoform', 'chrom', 'strand', 'length', 'exons', 'structural_category', 'associated_gene', 'CDS_length', 'CDS_start', 'CDS_end', 'CDS_genomic_start', 'CDS_genomic_end', 'predicted_NMD', 'gene_name'

In [10]:
adata_iso=adata_iso[:,adata_iso.var.dropna().index]

In [11]:
adata_iso.var['structural_category'].value_counts()

full-splice_match          18958
novel_not_in_catalog        4665
incomplete-splice_match     2214
novel_in_catalog             887
genic                        315
Name: structural_category, dtype: int64

In [12]:
adata_iso=adata_iso[:,adata_iso.var['structural_category'].isin(["full-splice_match","novel_not_in_catalog","incomplete-splice_match","novel_in_catalog"])]

In [13]:
adata_iso

View of AnnData object with n_obs × n_vars = 2999 × 26724
    obs: 'batch'
    var: 'isoform', 'chrom', 'strand', 'length', 'exons', 'structural_category', 'associated_gene', 'CDS_length', 'CDS_start', 'CDS_end', 'CDS_genomic_start', 'CDS_genomic_end', 'predicted_NMD', 'gene_name'

In [14]:
sc.pp.filter_genes(adata_iso,min_cells=3)

/usr/local/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number


In [15]:
sc.pp.filter_cells(adata_iso,min_genes=10)

In [16]:
adata_iso

AnnData object with n_obs × n_vars = 2999 × 15247
    obs: 'batch', 'n_genes'
    var: 'isoform', 'chrom', 'strand', 'length', 'exons', 'structural_category', 'associated_gene', 'CDS_length', 'CDS_start', 'CDS_end', 'CDS_genomic_start', 'CDS_genomic_end', 'predicted_NMD', 'gene_name', 'n_cells'

### Generate gene-level adata

In [17]:
adata_gene=scc.generate_Gene_adata(adata_iso,var_name="gene_name")

In [18]:
adata_gene

AnnData object with n_obs × n_vars = 2999 × 9697
    obs: 'batch', 'n_genes'

In [19]:
adata_gene.var

""
gene_name
ACBD3
ADGRL2
AK6
AL732488.2
AL844518.1
...
zwilch
zyg11
zyx


### Generate IF-level adata

In [20]:
adata_IF=scc.generate_IF_adata(adata_iso,var_name="gene_name")

Process successful for 0
Process successful for 1000
Process successful for 2000
Process successful for 3000
Process successful for 4000
Process successful for 5000
Process successful for 6000
Process successful for 7000
Process successful for 8000
Process successful for 9000
Process successful for 10000
Process successful for 11000
Process successful for 12000
Process successful for 13000
Process successful for 14000
Process successful for 15000


In [21]:
adata_IF

AnnData object with n_obs × n_vars = 2999 × 15247
    obs: 'batch', 'n_genes'
    var: 'isoform', 'chrom', 'strand', 'length', 'exons', 'structural_category', 'associated_gene', 'CDS_length', 'CDS_start', 'CDS_end', 'CDS_genomic_start', 'CDS_genomic_end', 'predicted_NMD', 'gene_name', 'n_cells'

### Generate PSI-level adata

In [22]:
event_path="../test/input/event.ioe"

In [23]:
adata_psi=scc.generate_PSI_adata(adata_iso,event_path)

/data/work/tools/scCyclone/read.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[column] = alternative_transcripts_number / total_transcripts_number
/usr/local/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [24]:
adata_psi

AnnData object with n_obs × n_vars = 2999 × 20963
    obs: 'batch', 'n_genes'
    var: 'gene_id', 'type', 'alternative_transcripts', 'total_transcripts'

In [26]:
adata_psi.var

,gene_id,type,alternative_transcripts,total_transcripts
ENSDARG00000103929;A3:1:45550-45788:45524-45788:-,ENSDARG00000103929,A3,ENSDART00000168428,"ENSDART00000168428,transcript2315.1.nnic"
ENSDARG00000103929;A3:1:48584-48890:48576-48890:-,ENSDARG00000103929,A3,"ENSDART00000168428,transcript1465.1.nnic,trans...","ENSDART00000168428,transcript1465.1.nnic,ENSDA..."
ENSDARG00000103929;A3:1:48584-48890:48564-48890:-,ENSDARG00000103929,A3,"ENSDART00000168428,transcript1465.1.nnic,trans...","ENSDART00000168428,ENSDART00000161565,transcri..."
ENSDARG00000103929;A3:1:46371-47421:45550-47421:-,ENSDARG00000103929,A3,transcript1471.1.nic,"transcript1465.1.nnic,transcript1471.1.nic"
ENSDARG00000103929;A3:1:48576-48890:48564-48890:-,ENSDARG00000103929,A3,ENSDART00000171162,"ENSDART00000161565,ENSDART00000171162"
...,...,...,...,...
ENSDARG00000097236;SE:25:35351925-35355335:35355372-35358145:-,ENSDARG00000097236,SE,ENSDART00000171917,"transcript42417.25.nnic,ENSDART00000154053,ENS..."
ENSDARG00000045554;SE:25:35960453-35961063:35961134-35963130:-,ENSDARG00000045554,SE,"ENSDART00000153612,ENSDART00000157334","ENSDART00000153919,ENSDART00000153612,ENSDART0..."
ENSDARG00000045636;SE:25:36031526-36031618:36031746-36040206:-,ENSDARG00000045636,SE,ENSDART00000073432,"ENSDART00000073432,ENSDART00000182207"
ENSDARG00000061282;SE:25:37223601-37223741:37223842-37224165:-,ENSDARG00000061282,SE,"ENSDART00000087247,ENSDART00000156647","ENSDART00000087247,ENSDART00000154045,ENSDART0..."


## DTU analysis

In [27]:
adata_iso

AnnData object with n_obs × n_vars = 2999 × 15247
    obs: 'batch', 'n_genes'
    var: 'isoform', 'chrom', 'strand', 'length', 'exons', 'structural_category', 'associated_gene', 'CDS_length', 'CDS_start', 'CDS_end', 'CDS_genomic_start', 'CDS_genomic_end', 'predicted_NMD', 'gene_name', 'n_cells'

In [28]:
import random

categories = ['A', 'B']

num_samples = adata_iso.shape[0]
random_categories = [random.choice(categories) for _ in range(num_samples)]

In [29]:
adata_iso.obs['label'] = random_categories
adata_iso.obs['label'] = adata_iso.obs['label'].astype("category")

In [30]:
scc.tl.rank_ifs_groups(adata_iso,groupby="label")

/usr/local/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number


Group A start!
Generate IF matrix...
Generate rank matrix...


/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:251: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  data_rank = pd.concat(data_rank_list, 0)
/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:251: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  data_rank = pd.concat(data_rank_list, 0)


Generate IF adata...
Process successful for 0


/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_IF[j] = list(sub_data / sub_gene)
/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_IF[j] = list(sub_data / sub_gene)
/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

...

/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:251: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  data_rank = pd.concat(data_rank_list, 0)
/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:251: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  data_rank = pd.concat(data_rank_list, 0)


Generate IF adata...
Process successful for 0


/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_IF[j] = list(sub_data / sub_gene)
/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_IF[j] = list(sub_data / sub_gene)
/data/work/tools/scCyclone/tools/_rank_ifs_groups.py:198: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

Process successful for 0
Compute dIF...
Compute pvalue...
Compute proportion...
Group B complete!
-----------------------------------------


AnnData object with n_obs × n_vars = 2999 × 15247
    obs: 'batch', 'n_genes', 'label'
    var: 'isoform', 'chrom', 'strand', 'length', 'exons', 'structural_category', 'associated_gene', 'CDS_length', 'CDS_start', 'CDS_end', 'CDS_genomic_start', 'CDS_genomic_end', 'predicted_NMD', 'gene_name', 'n_cells'
    uns: 'rank_ifs_groups'

In [32]:
scc.get.rank_ifs_groups_df(adata_iso)

,group,names,dif,dr,dr_state,dr_first,pvals,pvals_adj,dpr,gene_name
0,A,transcript39801.1.nnic,0.001003,"(1, 1)",normal,True,0.181273,1.0,-0.014146,cldng
1,A,ENSDART00000047159,0.009441,"(1, 1)",normal,True,0.047584,1.0,-0.022071,hmgb2a
2,A,ENSDART00000074689,0.032053,"(1, 1)",normal,True,0.457061,1.0,0.009537,eif5b
3,A,ENSDART00000084174,0.007793,"(1, 1)",normal,True,0.417753,1.0,0.010475,lig1
4,A,transcript69965.2.nnic,0.029608,"(2, 3)",up,False,0.167541,1.0,0.011663,lig1
...,...,...,...,...,...,...,...,...,...,...
145,B,ENSDART00000138350,0.018797,"(1, 1)",normal,True,0.265724,1.0,0.012588,rps12
146,B,ENSDART00000188105,0.002582,"(1, 1)",normal,True,0.973672,1.0,-0.000002,hmgb2b
147,B,transcript64375.23.nnic,0.055941,"(1, 1)",normal,True,0.061627,1.0,0.024923,psip1b
148,B,transcript35335.25.nic,0.034969,"(2, 2)",normal,False,0.644348,1.0,-0.004831,eif3ja


In [ ]:
switch_data=scc.tl.rank_switchs_groups(adata_iso)

In [ ]:
switch_data_info,switch_data_summary=scc.tl.rank_switch_consequences_groups(adata,switch_data,var_name_list=['exons','polyA_motif_found'])

## DPSI analysis

In [36]:
scc.tl.rank_psis_groups(adata_psi,groupby="label")

['A', 'B']


/usr/local/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/usr/local/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:251: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/usr/local/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.wa

Filter event: 20895
Group A start!
Compute dpsi...
Compute pvalue...
Group A complete!
-----------------------------------------
Group B start!
Compute dpsi...
Compute pvalue...
Group B complete!
-----------------------------------------


AnnData object with n_obs × n_vars = 2999 × 20963
    obs: 'batch', 'n_genes', 'label'
    var: 'gene_id', 'type', 'alternative_transcripts', 'total_transcripts'
    uns: 'rank_psis_groups'

In [38]:
adata_psi

AnnData object with n_obs × n_vars = 2999 × 20963
    obs: 'batch', 'n_genes', 'label'
    var: 'gene_id', 'type', 'alternative_transcripts', 'total_transcripts'
    uns: 'rank_psis_groups'

In [39]:
scc.get.rank_psis_groups_df(adata_psi,gene_symbols="gene_id",min_dpsi=0.1)

,group,names,dpsi,pvals,pvals_adj,gene_id
0,A,ENSDARG00000037713;A3:1:7557049-7559800:755704...,1.000,1.00,1.0,ENSDARG00000037713
1,A,ENSDARG00000061901;A3:5:19940575-19943507:1994...,1.000,1.00,1.0,ENSDARG00000061901
2,A,ENSDARG00000086150;A3:7:20918172-20919896:2091...,1.000,1.00,1.0,ENSDARG00000086150
3,A,ENSDARG00000062423;A3:9:24209707-24209910:2420...,1.000,1.00,1.0,ENSDARG00000062423
4,A,ENSDARG00000098983;A3:12:49000563-49003489:490...,1.000,1.00,1.0,ENSDARG00000098983
...,...,...,...,...,...,...
679,B,ENSDARG00000003058;SE:20:30589472-30594119:305...,0.111,1.00,1.0,ENSDARG00000003058
680,B,ENSDARG00000017439;SE:23:10434848-10435175:104...,0.109,1.00,1.0,ENSDARG00000017439
681,B,ENSDARG00000028335;AF:23:3758463-3759278:37593...,0.107,0.03,1.0,ENSDARG00000028335
682,B,ENSDARG00000057556;AF:21:19061766:19061998-190...,0.100,0.00,0.0,ENSDARG00000057556


In [40]:
event_list=list(adata_psi.var.index[:100])

In [ ]:
event_modal_data,event_best_modal_data=scc.get.psis_modal_df(adata_psi,groupby="label",groups=["A"],event_list=event_list,valid_cells=10)

In [44]:
event_modal_data

,ENSDARG00000103929;A3:1:45550-45788:45524-45788:-,ENSDARG00000103929;A3:1:48584-48890:48576-48890:-,ENSDARG00000103929;A3:1:48584-48890:48564-48890:-,ENSDARG00000014313;A3:1:613192-614504:613189-614504:-,ENSDARG00000037746;A3:1:8651631-8652579:8651610-8652579:-,ENSDARG00000063169;A3:1:9276450-9277168:9276437-9277168:-,ENSDARG00000089930;A3:1:11877213-11877367:11877210-11877367:-,ENSDARG00000056504;A3:1:16676019-16676110:16676015-16676110:-,ENSDARG00000006434;A3:1:19637622-19639781:19637599-19639781:-,ENSDARG00000037276;A3:1:25749687-25749769:25749657-25749769:-,...,ENSDARG00000093003;A3:1:52501962-52503591:52501948-52503591:-,ENSDARG00000024681;A3:1:55161475-55162440:55161424-55162440:-,ENSDARG00000099635;A3:1:59228465-59228550:59228459-59228550:-,ENSDARG00000101291;A3:1:59310440-59310594:59310437-59310594:-,ENSDARG00000101291;A3:1:59311097-59312061:59311091-59312061:-,ENSDARG00000102407;A3:1:12953-13034:12953-13070:+,ENSDARG00000102407;A3:1:15137-15352:15137-15365:+,ENSDARG00000063385;A3:1:278432-278719:278432-278728:+,ENSDARG00000074031;A3:1:6238776-6240431:6238776-6240437:+,ENSDARG00000058471;A3:1:9155305-9155420:9155305-9155440:+
bimodal,0.87,0.0,0.0,0.58,0.0,0.55,0.12,0.03,0.43,0.0,...,0.0,0.0,0.39,0.01,0.0,0.0,0.0,0.0,0.0,0.0
excluded,0.12,0.0,0.0,0.01,1.0,0.39,0.59,0.01,0.56,1.0,...,0.0,0.0,0.01,0.24,0.0,0.0,0.0,0.0,1.0,1.0
included,0.01,1.0,1.0,0.32,0.0,0.01,0.11,0.89,0.01,0.0,...,1.0,1.0,0.57,0.22,1.0,1.0,1.0,1.0,0.0,0.0
middle,0.00,0.0,0.0,0.09,0.0,0.05,0.18,0.07,0.00,0.0,...,0.0,0.0,0.03,0.53,0.0,0.0,0.0,0.0,0.0,0.0
